In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from itertools import product

In [2]:
Factory = ['Troy, NY', 'Newark, NJ', 'Harrisburg, PA']
Line = ['line 1', 'line 2', 'line 3']
Month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Model 1

In [3]:
m = gp.Model("All same")

Restricted license - for non-production use only - expires 2023-10-25


In [4]:
# Transportation cost
ship = pd.read_excel('Project Data.xlsx', 'shipping_cost')
ship = ship.iloc[:, 1:]
c = {(facility, customer): ship.iloc[facility, customer]
            for facility in range(0, 3)
            for customer in range(0, 8)}

# Production rate of production line l at factory i (Units per hour) 
wear = pd.read_excel('Project Data.xlsx', 'production_rate')
wear = wear.iloc[:, 1:]
w = {(facility, line): wear.iloc[facility, line]
            for facility in range(0, 3)
            for line in range(0, 3)}

# Production capacity : M

M = {(facility, line): wear.iloc[facility, line] * 160
            for facility in range(0, 3)
            for line in range(0, 3)}
# energy cost
energy = pd.read_excel('Project Data.xlsx', 'energy_cost')
energy = energy.iloc[:, 1:]
e = {(facility, line): energy.iloc[facility, line]
            for facility in range(0, 3)
            for line in range(0, 3)}
# demand
demand = pd.read_excel('Project Data.xlsx', 'demand')
demand = demand.iloc[:, 1:]
d = {(month, customer): demand.iloc[month, customer]
            for month in range(0, 12)
            for customer in range(0, 8)}

### Decision variable

In [5]:
# the number of units transported from factory i to store j in month m
u = m.addVars(list(product(range(0, 3), range(0, 8), range(0, 12))),lb = 0 ,vtype = GRB.INTEGER, name='units transported')

# the number of units produced at factory i from production line l in month m
s = m.addVars(list(product(range(0, 3), range(0, 3), range(0, 12))),lb = 0,vtype = GRB.INTEGER, name='units produced')

### Objective function

In [6]:
m.setObjective(( gp.quicksum(1.2 *c[i,j] * u[i, j, m] + e[i, l] * s[i, l, m] for l in range(0, 3) 
                             for m in range(0, 12) for j in range(0, 8) for i in range(0, 3))), GRB.MINIMIZE)

### Constractions

In [7]:
# 1. Demand Fulfilment on Transportation
m.addConstrs((gp.quicksum(u[i, j, m] for i in range(0, 3)) >= d[m, j] 
              for j in range(0, 8) for m in range(0, 12)), name='Demand')

# 2. Production Capacity
m.addConstrs((s[i, l, m] <= M[i, l] 
              for l in range(0, 3) for i in range(0, 3) for m in range(0, 12)), name='Capacity')

# 3. Demand Fulfilment on Factory
m.addConstrs((gp.quicksum(s[i, l, m] for i in range(0, 3) for l in range(0, 3)) >= (gp.quicksum(d[m, j] for j in range(0, 8)))
              for m in range(0, 12)), name='Demand')

# 4. Transportation units can’t exceed production units
m.addConstrs((gp.quicksum(u[i, j, m] for j in range(0, 8)) <= gp.quicksum(s[i, l, m] for l in range(0, 3))
              for i in range(0, 3) for m in range(0, 12)), name='Transport units')

# 5. Consistent production
m.addConstrs((gp.quicksum(s[0, l, m] for l in range(0, 3)) == gp.quicksum(s[1, l, m] for l in range(0, 3))
              for m in range(0, 12)), name = 'Consist production 1')


m.addConstrs((gp.quicksum(s[1, l, m] for l in range(0, 3)) == gp.quicksum(s[2, l, m] for l in range(0, 3))
              for m in range(0, 12)), name = 'Consist production 2')

# 6. Equal wear and tear
m.addConstrs((s[i, 0, m] / w[i, 0] == s[i, 1, m] / w[i, 1] 
              for i in range(0, 3) for m in range(0, 12)), name = '1')
m.addConstrs((s[i, 1, m] / w[i, 1] == s[i, 2, m] / w[i, 2] 
              for i in range(0, 3) for m in range(0, 12)), name = '2')


# Optimization
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 348 rows, 396 columns and 1188 nonzeros
Model fingerprint: 0xbf7bcb1d
Variable types: 0 continuous, 396 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+00]
  Objective range  [9e-02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+03]
Presolve removed 337 rows and 371 columns
Presolve time: 0.02s
Presolved: 11 rows, 25 columns, 51 nonzeros
Variable types: 0 continuous, 25 integer (1 binary)
Found heuristic solution: objective 2748831.7747
Found heuristic solution: objective 2747837.5066

Root relaxation: objective 2.747800e+06, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0          -    0      2747837.51 2747800.47  0

In [8]:
obj = m.getObjective()
model1_price = obj.getValue()
model1_price

2747837.506560001

In [9]:
# Output
factory = []
production_line = []
month = []
quantity = []
for facility in s.keys():
    factory.append(Factory[facility[0]])
    production_line.append(Line[facility[1]])
    month.append(Month[facility[2]])
    quantity.append(s[facility].x)

model1 = pd.DataFrame({'Factory':factory, 'ProductionLine':production_line,'Month':month, 'Quantity':quantity})
model1.to_excel('Model1_same_quantity.xlsx', index = 0)


In [10]:
model1

,Factory,ProductionLine,Month,Quantity
0,"Troy, NY",line 1,Jan,480.0
1,"Troy, NY",line 1,Feb,480.0
2,"Troy, NY",line 1,Mar,480.0
3,"Troy, NY",line 1,Apr,480.0
4,"Troy, NY",line 1,May,480.0
...,...,...,...,...
103,"Harrisburg, PA",line 3,Aug,48.0
104,"Harrisburg, PA",line 3,Sep,48.0
105,"Harrisburg, PA",line 3,Oct,48.0
106,"Harrisburg, PA",line 3,Nov,48.0


# Model 2

In [11]:
m = gp.Model("Difference")

In [12]:
# Transportation cost
# shipping_cost
ship = pd.read_excel('Project Data.xlsx', 'shipping_cost')
ship = ship.iloc[:, 1:]
c = {(facility, customer): ship.iloc[facility, customer]
            for facility in range(0, 3)
            for customer in range(0, 8)}
# Production rate of production line l at factory i (Units per hour) 
wear = pd.read_excel('Project Data.xlsx', 'production_rate')
wear = wear.iloc[:, 1:]
w = {(facility, line): wear.iloc[facility, line]
            for facility in range(0, 3)
            for line in range(0, 3)}
# Production capacity : M

M = {(facility, line): wear.iloc[facility, line] * 160
            for facility in range(0, 3)
            for line in range(0, 3)}
# energy cost
energy = pd.read_excel('Project Data.xlsx', 'energy_cost')
energy = energy.iloc[:, 1:]
e = {(facility, line): energy.iloc[facility, line]
            for facility in range(0, 3)
            for line in range(0, 3)}
# demand
demand = pd.read_excel('Project Data.xlsx', 'demand')
demand = demand.iloc[:, 1:]
d = {(month, customer): demand.iloc[month, customer]
            for month in range(0, 12)
            for customer in range(0, 8)}

### Decision variable

In [13]:
# the number of units transported from factory i to store j in month m
u = m.addVars(list(product(range(0, 3), range(0, 8), range(0, 12))),lb = 0 ,vtype = GRB.INTEGER, name='units transported')

# the number of units produced at factory i from production line l in month m
s = m.addVars(list(product(range(0, 3), range(0, 3), range(0, 12))),lb = 0,vtype = GRB.INTEGER, name='units produced')

### Objective function

In [14]:
m.setObjective((gp.quicksum(1.2 * c[i,j] * u[i, j, m] + e[i, l] * s[i, l, m] for l in range(0, 3) 
                             for m in range(0, 12) for j in range(0, 8) for i in range(0, 3))), GRB.MINIMIZE)

### Constractions

In [15]:
# 1. Demand Fulfilment on Transportation
m.addConstrs((gp.quicksum(u[i, j, m] for i in range(0, 3)) >= d[m, j] 
              for j in range(0, 8) for m in range(0, 12)), name='Demand')

# 2. Production Capacity
m.addConstrs((s[i, l, m] <= M[i, l] 
              for l in range(0, 3) for i in range(0, 3) for m in range(0, 12)), name='Capacity')

# 3. Demand Fulfilment on Factory
m.addConstrs((gp.quicksum(s[i, l, m] for i in range(0, 3) for l in range(0, 3)) >= (gp.quicksum(d[m, j] 
            for j in range(0, 8))) for m in range(0, 12)), name='Demand')

# 4. Transportation units can’t exceed production units
m.addConstrs((gp.quicksum(u[i, j, m] for j in range(0, 8)) <= gp.quicksum(s[i, l, m] for l in range(0, 3))
              for i in range(0, 3) for m in range(0, 12)), name='Transport units')

# 6. Equal wear and tear
m.addConstrs((s[i, 0, m] / w[i, 0] == s[i, 1, m] / w[i, 1] 
              for i in range(0, 3) for m in range(0, 12)), name = '1')

m.addConstrs((s[i, 1, m] / w[i, 1] == s[i, 2, m] / w[i, 2] 
              for i in range(0, 3) for m in range(0, 12)), name = '2')

m.addConstrs((s[i, 0, m] / w[i, 0] == s[i, 2, m] / w[i, 2] 
              for i in range(0, 3) for m in range(0, 12)), name = '3')

# m.addConstrs((s[0, l, m] / w[0, l] == s[1, l, m] / w[1, l] for l in range(0, 3) for m in range(0, 12)), name = 'Each factory all same 1')
# m.addConstrs((s[1, l, m] / w[1, l] == s[2, l, m] / w[2, l] for l in range(0, 3) for m in range(0, 12)), name = 'Each factory all same 2')

# Run optimization
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 360 rows, 396 columns and 1116 nonzeros
Model fingerprint: 0xee3ecdab
Variable types: 0 continuous, 396 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+00]
  Objective range  [9e-02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+03]
Found heuristic solution: objective 3664885.3824
Presolve removed 348 rows and 369 columns
Presolve time: 0.02s
Presolved: 12 rows, 27 columns, 54 nonzeros
Found heuristic solution: objective 1565689.7434
Variable types: 0 continuous, 27 integer (0 binary)
Found heuristic solution: objective 1565684.3520

Root relaxation: objective 1.504731e+06, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [16]:
obj = m.getObjective()
model2_price = obj.getValue()
model2_price

1505199.5673599998

In [17]:
# Output
factory = []
production_line = []
month = []
quantity = []
for facility in s.keys():
    factory.append(Factory[facility[0]])
    production_line.append(Line[facility[1]])
    month.append(Month[facility[2]])
    quantity.append(s[facility].x)

model2 = pd.DataFrame({'Factory':factory, 'ProductionLine':production_line,'Month':month, 'Quantity':quantity})
model2.to_excel('Model2_dif_quantity.xlsx', index = 0)

In [18]:
model2

,Factory,ProductionLine,Month,Quantity
0,"Troy, NY",line 1,Jan,150.0
1,"Troy, NY",line 1,Feb,186.0
2,"Troy, NY",line 1,Mar,228.0
3,"Troy, NY",line 1,Apr,252.0
4,"Troy, NY",line 1,May,168.0
...,...,...,...,...
103,"Harrisburg, PA",line 3,Aug,79.0
104,"Harrisburg, PA",line 3,Sep,79.0
105,"Harrisburg, PA",line 3,Oct,80.0
106,"Harrisburg, PA",line 3,Nov,80.0


# Model 3

In [19]:
m = gp.Model("model 3")

In [20]:
# Transportation cost
# shipping_cost
ship = pd.read_excel('Project Data.xlsx', 'shipping_cost')
ship = ship.iloc[:, 1:]
c = {(facility, customer): ship.iloc[facility, customer]
            for facility in range(0, 3)
            for customer in range(0, 8)}
# Production rate of production line l at factory i (Units per hour) 
w = {(facility, line): 6
            for facility in range(0, 3)
            for line in range(0, 3)}
# Production capacity : M

M = {(facility, line): 6 * 160
            for facility in range(0, 3)
            for line in range(0, 3)}
# energy cost
energy = pd.read_excel('Project Data.xlsx', 'energy_cost_new')
energy = energy.iloc[:, 1:]
e = {(facility, line): energy.iloc[0,line]
            for facility in range(0, 3)
            for line in range(0, 3)}
# demand
demand = pd.read_excel('Project Data.xlsx', 'demand')
demand = demand.iloc[:, 1:]
d = {(month, customer): demand.iloc[month, customer]
            for month in range(0, 12)
            for customer in range(0, 8)}

### Decision variable

In [21]:
# the number of units transported from factory i to store j in month m
u = m.addVars(list(product(range(0, 3), range(0, 8), range(0, 12))),lb = 0 ,vtype = GRB.INTEGER, name = 'units transported')

# the number of units produced at factory i from production line l in month m
s = m.addVars(list(product(range(0, 3), range(0, 3), range(0, 12))),lb = 0,vtype = GRB.INTEGER, name = 'units produced')

# Inventory of store j in month m
ii = m.addVars(list(product(range(0, 8), range(0, 12))),lb = 0,vtype = GRB.INTEGER, name = 'inventory')

### Objective function

In [22]:
m.setObjective((3 * gp.quicksum(1.2 * c[i,j] * u[i, j, m] + e[i, l] * s[i, l, m] for l in range(0, 3) 
                             for m in range(0, 12) for j in range(0, 8) for i in range(0, 3)) 
               ), GRB.MINIMIZE)

### Constractions

In [23]:
# 1. Demand Fulfilment on Transportation
m.addConstrs((gp.quicksum(u[i, j, m] for i in range(0, 3)) >= d[m, j] 
              for j in range(0, 8) for m in range(0, 12)), name='Demand')

# 2. Production Capacity
m.addConstrs((s[i, l, m] <= M[i, l] 
              for l in range(0, 3) for i in range(0, 3) for m in range(0, 12)), name='Capacity')

# 3. Demand Fulfilment on Factory
m.addConstrs((gp.quicksum(s[i, l, m] for i in range(0, 3) for l in range(0, 3)) >= (gp.quicksum(d[m, j] 
            for j in range(0, 8))) for m in range(0, 12)), name='Demand')

# 4. Transportation units can’t exceed production units
m.addConstrs((gp.quicksum(u[i, j, m] for j in range(0, 8)) <= gp.quicksum(s[i, l, m] for l in range(0, 3))
              for i in range(0, 3) for m in range(0, 12)), name='Transport units')

# 6. Equal wear and tear
m.addConstrs((s[i, 0, m] / w[i, 0] == s[i, 1, m] / w[i, 1] 
              for i in range(0, 3) for m in range(0, 12)), name = '1')

m.addConstrs((s[i, 1, m] / w[i, 1] == s[i, 2, m] / w[i, 2] 
              for i in range(0, 3) for m in range(0, 12)), name = '2')

m.addConstrs((s[i, 0, m] / w[i, 0] == s[i, 2, m] / w[i, 2] 
              for i in range(0, 3) for m in range(0, 12)), name = '3')

# m.addConstrs((s[0, l, m] / w[0, l] == s[1, l, m] / w[1, l] for l in range(0, 3) for m in range(0, 12)), name = 'Each factory all same 1')
# m.addConstrs((s[1, l, m] / w[1, l] == s[2, l, m] / w[2, l] for l in range(0, 3) for m in range(0, 12)), name = 'Each factory all same 2')

# Run optimization
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 360 rows, 492 columns and 1116 nonzeros
Model fingerprint: 0x2b9ffc21
Variable types: 0 continuous, 492 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [3e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+03]
Found heuristic solution: objective 8298118.9152
Presolve removed 348 rows and 465 columns
Presolve time: 0.01s
Presolved: 12 rows, 27 columns, 54 nonzeros
Found heuristic solution: objective 2438252.4096
Variable types: 0 continuous, 27 integer (0 binary)
Found heuristic solution: objective 2438248.0320

Root relaxation: objective 2.433444e+06, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [24]:
obj = m.getObjective()
model3_price = obj.getValue()
model3_price

2433444.05376

In [25]:
# Output
factory = []
production_line = []
month = []
quantity = []
for facility in s.keys():
    factory.append(Factory[facility[0]])
    production_line.append(Line[facility[1]])
    month.append(Month[facility[2]])
    quantity.append(s[facility].x)

model3 = pd.DataFrame({'Factory':factory, 'ProductionLine':production_line,'Month':month, 'Quantity':quantity})
model3.to_excel('Model3_quantity.xlsx', index = 0)

In [26]:
model3

,Factory,ProductionLine,Month,Quantity
0,"Troy, NY",line 1,Jan,117.0
1,"Troy, NY",line 1,Feb,125.0
2,"Troy, NY",line 1,Mar,150.0
3,"Troy, NY",line 1,Apr,158.0
4,"Troy, NY",line 1,May,150.0
...,...,...,...,...
103,"Harrisburg, PA",line 3,Aug,117.0
104,"Harrisburg, PA",line 3,Sep,84.0
105,"Harrisburg, PA",line 3,Oct,84.0
106,"Harrisburg, PA",line 3,Nov,92.0


In [27]:
# Reserve price

In [34]:
3*model1_price

8243512.519680003

In [35]:
3 * model1_price - model3_price

5810068.465920003

## Q1

In [36]:
model1_price

2747837.506560001

In [37]:
model2_price

1505199.5673599998

In [32]:
model1_price - model2_price

1242637.9392000013

### Q2

In [33]:
3 * model1_price - model3_price

5810068.465920003

In [38]:
model3_price

2433444.05376

In [39]:
3 * model1_price

8243512.519680003